<a href="https://colab.research.google.com/github/cedkim/deeplearning-courseenv/blob/master/4-tensorflow-audiobooks-example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practical example: Audiobooks

## Import the necessary libraries and download the files

In [1]:
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
!wget -O audiobooks_data.csv "https://onedrive.live.com/download?cid=22B0F983382BA466&resid=22B0F983382BA466%21299394&authkey=AKsawparTh_7fTc"

--2020-08-05 10:18:40--  https://onedrive.live.com/download?cid=22B0F983382BA466&resid=22B0F983382BA466%21299394&authkey=AKsawparTh_7fTc
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://hhf3tw.am.files.1drv.com/y4mJm0CP5GReziF-KDQhl0sjN0EuRHtplGJGXSxd5pb9co-UpLwk9KvUITdABcpNyzmbBcVyFqsTi8X8c1fi3ZnjoTt_cKegWLs2CSC0M0sAi4EnMqqzAVdJSwHovBurDJPr7BJTiQvGO3YrV9TOrK-kqgWIJ4xgJC0bVeW7KsQQKkRp1NSo_qtDu1e4N4BXRgl0UWh9nNcKsQz5S_VjVICfQ/1.1%20Audiobooks_data.csv.csv?download&psid=1 [following]
--2020-08-05 10:18:41--  https://hhf3tw.am.files.1drv.com/y4mJm0CP5GReziF-KDQhl0sjN0EuRHtplGJGXSxd5pb9co-UpLwk9KvUITdABcpNyzmbBcVyFqsTi8X8c1fi3ZnjoTt_cKegWLs2CSC0M0sAi4EnMqqzAVdJSwHovBurDJPr7BJTiQvGO3YrV9TOrK-kqgWIJ4xgJC0bVeW7KsQQKkRp1NSo_qtDu1e4N4BXRgl0UWh9nNcKsQz5S_VjVICfQ/1.1%20Audiobooks_data.csv.csv?download&psid=1
Resolving hhf3tw.am.file

## Preprocess the data

### Extract the data from the csv

In [2]:
raw_csv_data = np.loadtxt('audiobooks_data.csv', delimiter=',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

### Balance the dataset

In [3]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [4]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffle the data

In [5]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the dataset into train, validation, and test

In [6]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
test_targets = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1790.0 3579 0.5001397038278849
-8.344215963817142 447 -0.018667149807197185
-8.344215963817142 448 -0.018625482062091834


### Save the three datasets in *.npz

In [7]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

## Create the machine learning algorithm

### Data

In [8]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')

validation_inputs = npz['inputs'].astype(np.float)
validation_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')

test_inputs = npz['inputs'].astype(np.float)
test_targets = npz['targets'].astype(np.int)

### Model

#### Outline

In [23]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

#### Optimizers

In [24]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#### Training (with early stopping)

In [26]:
batch_size = 100

max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
          train_targets,
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose=2)

Epoch 1/100
27/36 [=====================>........] - ETA: 0s - loss: 0.3975 - accuracy: 0.8641

ValueError: ignored

### Testing the model

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)